# Gaming Analytics – In-Game Monetization & Paying User Behavior 🎮💰

**Author:** Luis Chaumer  
**Role:** Data Analyst  

This project analyzes **in-game monetization** for a fictional free-to-play mobile game.

Using synthetic datasets for **20,000 users** and their **in-app purchases**, we:
- Measure key monetization KPIs (ARPU, ARPPU, payer conversion, revenue per country & channel)
- Analyze the distribution of spending (whales, dolphins, minnows)
- Study how acquisition channels and countries affect revenue
- Use **Python + SQL (SQLite)** to replicate real gaming analytics workflows
- Provide insights and recommendations to improve monetization strategy

Datasets:
- `data/gaming_users_dataset.csv`
- `data/gaming_purchases_dataset.csv`


## 1. Setup: imports and configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

plt.style.use("default")
pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

## 2. Load datasets

In [ ]:
users_path = "data/gaming_users_dataset.csv"
purchases_path = "data/gaming_purchases_dataset.csv"

users = pd.read_csv(users_path, parse_dates=["install_date"])
purchases = pd.read_csv(purchases_path, parse_dates=["purchase_datetime"])

users.head(), purchases.head()

## 3. Basic exploration

In [ ]:
users.shape, purchases.shape

In [ ]:
users.head()

In [ ]:
purchases.head()

In [ ]:
users['is_payer'].value_counts(normalize=True).rename('payer_conversion')

## 4. Monetization KPIs

In [ ]:
# Aggregate revenue per user
revenue_per_user = purchases.groupby("user_id")["amount"].sum().rename("revenue")
users_monetization = users.merge(revenue_per_user, on="user_id", how="left")
users_monetization["revenue"] = users_monetization["revenue"].fillna(0)

total_revenue = users_monetization["revenue"].sum()
n_users = len(users_monetization)
n_payers = (users_monetization["revenue"] > 0).sum()

arpu = total_revenue / n_users
arppu = total_revenue / n_payers
payer_conversion = n_payers / n_users

pd.Series({
    "Total revenue": total_revenue,
    "Number of users": n_users,
    "Number of payers": n_payers,
    "Payer conversion": payer_conversion,
    "ARPU": arpu,
    "ARPPU": arppu
})

### 4.1 Revenue distribution

In [ ]:
plt.figure(figsize=(6,4))
users_monetization["revenue"].clip(upper=200).hist(bins=40)
plt.title("Revenue per user (clipped at 200)")
plt.xlabel("Revenue")
plt.ylabel("Number of users")
plt.tight_layout()
plt.show()

## 5. Whales, dolphins, minnows

In [ ]:
# Define spending segments among payers only
payers = users_monetization[users_monetization["revenue"] > 0].copy()

quantiles = payers["revenue"].quantile([0.5, 0.9, 0.99])
quantiles

In [ ]:
def classify_spender(x, q):
    if x <= q[0.5]:
        return "minnow"
    elif x <= q[0.9]:
        return "dolphin"
    else:
        return "whale"

payers["spender_segment"] = payers["revenue"].apply(classify_spender, q=quantiles)

segment_summary = payers.groupby("spender_segment").agg(
    users=("user_id", "nunique"),
    total_revenue=("revenue", "sum"),
    avg_revenue=("revenue", "mean")
)

segment_summary["share_of_users"] = segment_summary["users"] / segment_summary["users"].sum()
segment_summary["share_of_revenue"] = segment_summary["total_revenue"] / segment_summary["total_revenue"].sum()

segment_summary.sort_values("avg_revenue", ascending=True)

In [ ]:
plt.figure(figsize=(6,4))
segment_summary["share_of_revenue"].plot(kind="bar")
plt.title("Share of revenue by spender segment")
plt.ylabel("Share of revenue")
plt.tight_layout()
plt.show()

## 6. Revenue by country and acquisition channel

In [ ]:
country_rev = users_monetization.groupby("country")["revenue"].agg(["sum", "mean"]).rename(columns={"sum":"total_revenue","mean":"arpu"})
country_rev.sort_values("total_revenue", ascending=False)

In [ ]:
plt.figure(figsize=(8,4))
country_rev["total_revenue"].sort_values(ascending=False).plot(kind="bar")
plt.title("Total revenue by country")
plt.ylabel("Revenue")
plt.tight_layout()
plt.show()

In [ ]:
channel_rev = users_monetization.groupby("acquisition_channel")["revenue"].agg(["sum", "mean"]).rename(columns={"sum":"total_revenue","mean":"arpu"})
channel_rev.sort_values("total_revenue", ascending=False)

In [ ]:
plt.figure(figsize=(8,4))
channel_rev["arpu"].sort_values(ascending=False).plot(kind="bar")
plt.title("ARPU by acquisition channel")
plt.ylabel("ARPU")
plt.tight_layout()
plt.show()

## 7. SQL analysis with SQLite

We now use **SQL** (via SQLite) for classic gaming monetization analysis tasks:

- Revenue per country & platform  
- Payer conversion by acquisition channel  
- Top segments by ARPPU  

This replicates how analysts often explore data in BI / data warehouse environments.


In [ ]:
conn = sqlite3.connect(":memory:")
users_monetization.to_sql("users", conn, index=False, if_exists="replace")
purchases.to_sql("purchases", conn, index=False, if_exists="replace")

### 7.1 Revenue per country & platform (SQL)

In [ ]:
query = '''
SELECT 
    country,
    platform,
    COUNT(DISTINCT user_id) AS users,
    SUM(revenue) AS total_revenue,
    ROUND(AVG(revenue), 2) AS arpu
FROM users
GROUP BY country, platform
ORDER BY total_revenue DESC;
'''
sql_country_platform = pd.read_sql_query(query, conn)
sql_country_platform

### 7.2 Payer conversion by acquisition channel (SQL)

In [ ]:
query = '''
SELECT 
    acquisition_channel,
    COUNT(*) AS total_users,
    SUM(CASE WHEN revenue > 0 THEN 1 ELSE 0 END) AS payers,
    ROUND(1.0 * SUM(CASE WHEN revenue > 0 THEN 1 ELSE 0 END) / COUNT(*), 4) AS payer_conversion,
    ROUND(AVG(revenue), 2) AS arpu,
    ROUND(SUM(revenue) / NULLIF(SUM(CASE WHEN revenue > 0 THEN 1 ELSE 0 END), 0), 2) AS arppu
FROM users
GROUP BY acquisition_channel
ORDER BY payer_conversion DESC;
'''
sql_channel_conversion = pd.read_sql_query(query, conn)
sql_channel_conversion

### 7.3 Top whale countries (SQL)

In [ ]:
query = '''
SELECT 
    country,
    COUNT(*) AS payers,
    ROUND(AVG(revenue), 2) AS arppu,
    SUM(revenue) AS total_revenue
FROM users
WHERE revenue > 0
GROUP BY country
HAVING COUNT(*) >= 50
ORDER BY arppu DESC
LIMIT 10;
'''
sql_whale_countries = pd.read_sql_query(query, conn)
sql_whale_countries

## 8. Conclusions and recommendations

_Suggested narrative (to be adapted to actual results):_

- The game achieves an overall **payer conversion rate** of ~X%, with **ARPU** of ~Y and **ARPPU** of ~Z.  
- A small fraction of users ("whales") contributes a disproportionately high share of revenue, which is typical in F2P games.  
- Certain countries and acquisition channels show higher ARPU and ARPPU, suggesting more efficient monetization.  
- Organic users might have lower conversion but stable long-term engagement, while paid channels (e.g., Facebook / Google Ads) bring more high-value payers.  

**Business recommendations:**

1. Increase UA (user acquisition) budget on the most profitable channels (high ARPU / ARPPU, not just low CPI).  
2. Design in-game offers and events targeted to mid-spenders ("dolphins") to encourage upsell.  
3. Monitor whale behavior and churn risk, as their departure strongly impacts revenue.  
4. Localize offers and pricing by country to maximize conversion.  
5. Combine this monetization analysis with **retention and engagement metrics** in future iterations.  

This project demonstrates how to analyze **in-game monetization** using **Python, SQL, and standard F2P gaming KPIs**.
